<html>
<body>

<p><font size="5" color="orangered">
🎯 CSPNET: A NEW BACKBONE THAT CAN ENHANCE LEARNING CAPABILITY OF CNN
    
</font></p>
</body>
</html>

![](https://miro.medium.com/max/2814/1*h1ouOibp3Ry7vj4r16cyVQ.png)

<html>
<body>

<p><font size="5" color="darkcyan">

Neural networks have enabled state-of-the-art approaches to achieve incredible results on computer vision tasks such as object detection. However, such success greatly relies on costly computation resources, which hinders people with cheap devices from appreciating the advanced technology. 

Cross Stage Partial Network (CSPNet) help mitigate the problem that previous works require heavy inference computations from the network architecture perspective. This is attributed to the problem to the duplicate gradient information within network optimization. 

The proposed network respect the variability of the gradients by integrating feature maps from the beginning and the end of a network stage, which, in our experiments, reduces computations by 20% with equivalent or even superior accuracy on the ImageNet dataset.

The CSPNet is easy to implement and general enough to cope with architectures based on ResNet, ResNeXt, and DenseNet

🔥To find out more :- https://arxiv.org/pdf/1911.11929.pdf 🔥

</font></p>
</body>
</html>

<html>
<body>

<p><font size="6" color="dodgerblue">
👍🏻 Upvote the notebook if you find it insightful!
    
</font></p>
</body>
</html>


<html>
<body>

<p><font size="5" color="darkmagenta">
🪃 Fetch the required libraries
    
</font></p>
</body>
</html>


In [ ]:
# Add the timm pytorch image model library from which we can extract CSPNet
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
# Software library written for data manipulation and analysis.
import pandas as pd

# Python library to interact with the file system.
import os

# Python library for image augmentation
import albumentations as A

# fastai library for computer vision tasks
from fastai.vision.all import *

# Developing and training neural network based deep learning models.
import torch
from torch import nn

<html>
<body>

<p><font size="5" color="darkmagenta">
💉 Data Loading

Major credits to [Darek Kłeczek](https://www.kaggle.com/thedrcat) for providing this dataset!
    
</font></p>
</body>
</html>


In [ ]:
# Define path to dataset, whose benefit is that this sample is more balanced than original train data.
path = Path('../input/hpa-cell-tiles-sample-balanced-dataset')

In [ ]:
df = pd.read_csv(path/'cell_df.csv')
df.head()

In [ ]:
# extract the the total number of target labels
labels = [str(i) for i in range(19)]
for x in labels: df[x] = df['image_labels'].apply(lambda r: int(x in r.split('|')))

In [ ]:
# Here a sample of the dataset has been taken, change frac to 1 to train the entire dataset!
dfs = df.sample(frac=0.1, random_state=42)
dfs = dfs.reset_index(drop=True)
len(dfs)

<html>
<body>

<p><font size="5" color="darkmagenta">
📝 Data Preprocessing
    
</font></p>
</body>
</html>



In [ ]:
# obtain the input images.
def get_x(r): 
    return path/'cells'/(r['image_id']+'_'+str(r['cell_id'])+'.jpg')

# obtain the targets.
def get_y(r): 
    return r['image_labels'].split('|')

<html>
<body>

<p><font size="5" color="darkcyan">
🖨 Albumentations:
Albumentations is a Python library for image augmentation. Image augmentation is used in deep learning and computer vision tasks to increase the quality of trained models. The purpose of image augmentation is to create new training samples from the existing data.

Albumentations supports all common computer vision tasks such as classification, semantic segmentation, instance segmentation, object detection, and pose estimation.
The library provides a simple unified API to work with all data types: images (RBG-images, grayscale images, multispectral images), segmentation masks, bounding boxes, and keypoints.
The library contains more than 70 different augmentations to generate new training samples from the existing data.
Albumentations is fast.

Installation:- pip install -U albumentations
    
</font></p>
</body>
</html>


In [ ]:
'''AlbumentationsTransform will perform different transforms over both
   the training and validation datasets ''' 
class AlbumentationsTransform(RandTransform):
    
    '''split_idx is None, which allows for us to say when we're setting our split_idx.
       We set an order to 2 which means any resize operations are done first before our new transform. '''
    split_idx, order = None, 2
    
    def __init__(self, train_aug, valid_aug): store_attr()
    
    # Inherit from RandTransform, allows for us to set that split_idx in our before_call.
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    # If split_idx is 0, run the trainining augmentation, otherwise run the validation augmentation. 
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
def get_train_aug(size): 
    
    return A.Compose([
            # allows to combine RandomCrop and RandomScale
            A.RandomResizedCrop(size,size),
            
            # Transpose the input by swapping rows and columns.
            A.Transpose(p=0.5),
        
            # Flip the input horizontally around the y-axis.
            A.HorizontalFlip(p=0.5),
        
            # Flip the input horizontally around the x-axis.
            A.VerticalFlip(p=0.5),
        
            # Randomly apply affine transforms: translate, scale and rotate the input.
            A.ShiftScaleRotate(p=0.5),
        
            # Randomly change hue, saturation and value of the input image.
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        
            # Randomly change brightness and contrast of the input image.
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        
            # CoarseDropout of the rectangular regions in the image.
            A.CoarseDropout(p=0.5),
        
            # CoarseDropout of the square regions in the image.
            A.Cutout(p=0.5) ])

def get_valid_aug(size): 
    
    return A.Compose([
    # Crop the central part of the input.   
    A.CenterCrop(size, size, p=1.),
    
    # Resize the input to the given height and width.    
    A.Resize(size,size)], p=1.)

In [ ]:
'''The first step item_tfms resizes all the images to the same size (this happens on the CPU) 
   and then batch_tfms happens on the GPU for the entire batch of images. '''
# Transforms we need to do for each image in the dataset
item_tfms = [Resize(224), AlbumentationsTransform(get_train_aug(224), get_valid_aug(224))]

# Transforms that can take place on a batch of images
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

bs=6

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock(vocab=labels)), # multi-label target
                splitter=RandomSplitter(seed=42), # split data into training and validation subsets.
                get_x=get_x, # obtain the input images.
                get_y=get_y,  # obtain the targets.
                item_tfms=item_tfms,
                batch_tfms=batch_tfms
                )

dls = dblock.dataloaders(dfs, bs=bs)

In [ ]:
# We can call show_batch() to see what a sample of a batch looks like.
dls.show_batch()

<html>
<body>

<p><font size="6" color="darkmagenta">
⌛️Model Definition
    
</font></p>
</body>
</html>

<html>
<body>

<p><font size="5" color="darkcyan">
🛠Cross Stage Partial Network (CSPNet) 🛠<br>
    
The main purpose of designing CSPNet is to enable this architecture to achieve a richer gradient combination while reducing the amount of computation. This aim is achieved by partitioning feature map of the base layer into two parts and then merging them through a proposed cross-stage hierarchy. <br>
    
The main concept is to make the gradient flow propagate through different network paths
by splitting the gradient flow. In this way, we have confirmed that the propagated gradient information can have a large correlation difference by switching concatenation and transition steps. In addition, CSPNet can greatly reduce the amount of computation, and improve inference speed as well as accuracy. <br>

The proposed CSPNet deals with the following three problems: <br>
    
1) Strengthening learning ability of a CNN The accuracy of existing CNN is greatly degraded after lightweightening,so we CSPNet can maintain sufficient accuracy while being lightweightening.
The proposed CSPNet can be easily applied to ResNet, ResNeXt, and DenseNet. After applying CSPNet on the
above mentioned networks, the computation effort can be reduced from 10% to 20%, but it outperforms ResNet,
ResNeXt, DenseNet <br>

2) Removing computational bottlenecks Too high a computational bottleneck will result in more cycles to complete the inference process, or some arithmetic units will often idle. Therefore, we hope we can evenly distribute the amount of computation at each layer in CNN so that we can effectively upgrade the utilization rate of each computation unit and thus reduce unnecessary energy consumption. <br>
    
3) Reducing memory costs The wafer fabrication cost of Dynamic Random-Access Memory (DRAM) is very expensive,
and it also takes up a lot of space. If one can effectively reduce the memory cost, he/she will greatly reduce the cost of ASIC. In addition, a small area wafer can be used in a variety of edge computing devices. In reducing the use of memory usage, CSPNet adopt cross-channel pooling to compress the feature maps during the feature pyramid generating process. 
    
</font></p>
</body>
</html>



In [ ]:
class CSPNetModel(nn.Module):
    
    def __init__(self, num_classes=19, model_name='cspresnext50', pretrained=True):
        super(CSPNetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        self.model.head.fc = nn.Linear(self.model.head.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
model = CSPNetModel()


In [ ]:
# Group together some dls, a model, and metrics to handle training
learn = Learner(dls, model, metrics= accuracy_multi)

In [ ]:
# Choosing a good learning rate
learn.lr_find()

In [ ]:
# We can use the fine_tune function to train a model with this given learning rate
learn.fine_tune(9,0.0012022644514217973)

In [ ]:
# Plot training and validation losses.
learn.recorder.plot_loss()

In [ ]:
# Interpretation methods for classification models.
interp = ClassificationInterpretation.from_learner(learn)

# Show images in top_losses along with their prediction, actual, loss, and probability of actual class.
interp.plot_top_losses(5, nrows=5)

![](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_299233%2Fproject_400509%2Fimages%2Ffig%2Fio.png)


<html>
<body>

<p><font size="5" color="darkcyan">
🎉 The above figure shows the size of each layer of ResNeXt50 and the proposed CSPResNeXt50. The
CIO of the proposed CSPResNeXt (32.6M) is lower than that of the original ResNeXt50 (34.4M). In addition, the CSPResNeXt50 removes the bottleneck layers in the ResXBlock and maintains the same numbers of the input channel
and the output channel and will have the most efficient computation when FLOPs are fixed. The low CIO and FLOPs enable our CSPResNeXt50 to outperform the vanilla ResNeXt50 by 22% in terms of computations.
    
</font></p>
</body>
</html>

<html>
<body>

<p><font size="5" color="mediumvioletred">
🎊 That's it for today's review of one of the latest breakthroughs in Computer Vision!! 🎊<br><br>
🔥 Let me know in the comments below if you would like me to explain the latest innovations in deep learning in notebooks like these with implementation in fastai or pytorch 🔥
    
</font></p>
</body>
</html>